In [27]:
import sqlite3 as sq
import numpy as np
import pickle
import natsort
from scipy.optimize import curve_fit
from scipy.stats import linregress
from matplotlib import pyplot as plt
from multiprocessing.pool import Pool
%matplotlib inline
def get_table(conn, table):
    out = None
    with conn:
        out = conn.execute(f"select * from {table};").fetchall();
    out_dict = {k[0]: np.array(k[1:]) for k in out}
    return (get_tau(out), out_dict)

def get_tau(table):
    return np.array(range(1, len(table[0]))) if len(table) >= 1 else None

def plot(data, key, q):
    plt.plot(data[key][0], data[key][1][q])
conn = sq.connect("../crowd.sqlite")

def list_tables(conn, filter_str: str = ""):
    with conn:
        tables = conn.execute("SELECT name FROM main.sqlite_master WHERE type='table'").fetchall();
    return list(filter(lambda t: t.find(filter_str) != -1, map(lambda t: t[0], tables)))

In [30]:
all_boxsize_tables = list_tables(conn, "multiDDM")

In [31]:
#Sort into video files
#get tables
# fits as a function of box size
video_names = set(map(lambda t: '_'.join(t.split('_')[0:-1]), all_boxsize_tables))

In [32]:
tables = {k: natsort.natsorted(filter(lambda t: t.find(f"{k}_") != -1, tables)) for k in video_names}

In [38]:
data = {k: {int(e.split("_")[-1]): get_table(conn, e) for e in v} for k, v in tables.items()}

In [39]:
crowd_data = {k: v for k, v in data.items() if k.find("crowd") != -1}
running_data = {k: v for k, v in data.items() if k.find("running") != -1}

In [61]:
def brownian_isf(tau, tauc, a, b):
    return a * ( 1 - np.exp(-tau / tauc)) + b
def ballistic_isf(tau, tauc, vq, a, b):
    return a * (1 - np.sinc(vq * tau) * np.exp(- tau / tauc)) + b
brownian_bounds = ([0, -np.inf, -np.inf], [np.inf, np.inf, np.inf])
ballistic_bounds = ([0, -np.inf, -np.inf, -np.inf], [np.inf, np.inf, np.inf, np.inf])

In [88]:
def fit(*args):
    q, tau, I, f, bounds = args[0]
    try:
        fits, errs = curve_fit(f, tau, I, bounds=bounds)
        errs = np.sqrt(np.diag(errs))
    except RuntimeError:
        return None
    else:
        #for ballistic fits the fit is vq and not v as desired
        if len(fits) == 4:
            fits[1] /= q
        return fits, errs
        
def fit_all(data, f, bounds):
    pool = Pool(20)
    return {
        video: {
            box_size: (
                d[0], 
                pool.map(fit, [
                    (q, d[0], I, f, bounds) for q, I in d[1].items()
                ])
            ) for box_size, d in dat.items()
    } for video, dat in data.items()}

In [75]:
crowd_brownian_fit = fit_all(crowd_data, brownian_isf, ([0, -np.inf, -np.inf], [np.inf, np.inf, np.inf]))
crowd_ballistic_fit = fit_all(crowd_data, ballistic_isf, ([0, -np.inf, -np.inf, -np.inf], [np.inf, np.inf, np.inf, np.inf]))
running_brownian_fit = fit_all(running_data, brownian_isf, ([0, -np.inf, -np.inf], [np.inf, np.inf, np.inf]))
running_ballistic_fit = fit_all(running_data, ballistic_isf, ([0, -np.inf, -np.inf, -np.inf], [np.inf, np.inf, np.inf, np.inf]))

In [76]:
def save_pickle(data, output: str):
    with open(output, "wb") as f:
        pickle.dump(data, f)

In [93]:
def fix_formatting(data):
    return {video_name: {
        box_size: (d[0], {
            (2*i + 3)/2: (e[0], np.sqrt(np.diag(e[1]))) for i, e in enumerate(d[1]) if e is not None
        }) for box_size, d in dat.items()
    } for video_name, dat in data.items()}

In [94]:
crowd_brownian_fit_formatted = fix_formatting(crowd_brownian_fit)
crowd_ballistic_fit_formatted = fix_formatting(crowd_ballistic_fit)
running_brownian_fit_formatted = fix_formatting(running_brownian_fit)
running_ballistic_fit_formatted = fix_formatting(running_ballistic_fit)

In [95]:
save_pickle(crowd_brownian_fit_formatted, "crowd_brownian_fit.pickle")
save_pickle(crowd_ballistic_fit_formatted, "crowd_ballistic_fit.pickle")
save_pickle(running_brownian_fit_formatted, "running_brownian_fit.pickle")
save_pickle(running_ballistic_fit_formatted, "running_ballistic_fit.pickle")